In [1]:
# [1] 初期設定
import os   
import json
import base64
import requests
from pathlib import Path
from dotenv import load_dotenv
from tqdm import tqdm
from autoLeague.replays.scraper import ReplayScraper
from autoLeague.dataset.downloader import ReplayDownloader

# RiotAPI.envからAPIキーを読み込み
#load_dotenv("RiotAPI.env")
load_dotenv("RiotAPI.env")
API_KEY = os.environ.get("API_KEY")

# ── 共通パス設定 ─────────────────────────────────────
USER_HOME = Path(os.environ["USERPROFILE"])    # C:\Users\<アカウント名>

GAME_DIR      = Path(r"C:\Riot Games\League of Legends\Game")
REPLAY_DIR    = str(USER_HOME / "Documents" / "League of Legends" / "Replays")
SAVE_DIR      = r'C:\dataset_20260105'  # OneDrive競合回避のためDocuments外に配置
SCRAPER_DIR   = str(USER_HOME / "Desktop" / "LoL_WorkSp_win" / "pyLoL-_WorkSp" / "pyLoL-v2" / "autoLeague" / "replays")
PROGRESS_LOG  = os.path.join(SAVE_DIR, "progress.json")  # 処理進捗ログ

# 保存先フォルダ作成
os.makedirs(SAVE_DIR, exist_ok=True)

# ReplayScraper初期化
rs = ReplayScraper(
    game_dir=GAME_DIR, 
    replay_dir=REPLAY_DIR, 
    save_dir=SAVE_DIR, 
    scraper_dir=SCRAPER_DIR, 
    replay_speed=8, 
    region="JP1"
)

# ReplayDownloader初期化（LCU認証情報取得用）
rd = ReplayDownloader()

print(f"REPLAY_DIR: {REPLAY_DIR}")
print(f"SAVE_DIR: {SAVE_DIR}")
print(f"PROGRESS_LOG: {PROGRESS_LOG}")
print(f"LCU Port: {rd.port}")

LCU API initialized - port: 57328
Current number of replays:  190
remoting-auth-token : 38FfaxIF9OorCQ42BQaZug
REPLAY_DIR: C:\Users\lapis\Documents\League of Legends\Replays
SAVE_DIR: C:\dataset_20260105
PROGRESS_LOG: C:\dataset_20260105\progress.json
LCU Port: 57328


In [2]:
# [2] LCU API デバッグ - 接続状況とエンドポイント確認
# このセルでLCU APIへの接続状態を詳しく確認します

# LCU API認証ヘッダー作成
auth = 'Basic ' + base64.b64encode(f'riot:{rd.token}'.encode()).decode()
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': auth
}
requests.packages.urllib3.disable_warnings()

print(f"=== LCU API 接続情報 ===")
print(f"Port: {rd.port}")
print(f"Token: {rd.token[:20]}..." if rd.token else "Token: None")
print()

# 1. LCU APIの基本接続テスト
print("=== 接続テスト ===")
try:
    # システム情報取得（基本的なエンドポイント）
    url = f'https://127.0.0.1:{rd.port}/system/v1/builds'
    resp = requests.get(url, headers=headers, verify=False, timeout=5)
    print(f"System builds: {resp.status_code}")
    if resp.status_code == 200:
        print(f"  → LCU API接続OK")
except Exception as e:
    print(f"  → 接続エラー: {e}")

# 2. リプレイ関連エンドポイントの確認
print("\n=== リプレイAPIテスト ===")

# roflファイル一覧を取得
rofl_files = [f for f in os.listdir(REPLAY_DIR) if f.endswith('.rofl')]
print(f"REPLAY_DIR内のroflファイル数: {len(rofl_files)}")

if rofl_files:
    test_replay = rofl_files[0]
    game_id = test_replay.replace('.rofl', '')
    numeric_id = game_id.split('-')[-1]
    
    print(f"\nテスト対象: {test_replay}")
    print(f"Game ID: {game_id}")
    print(f"Numeric ID: {numeric_id}")
    
    # メタデータ取得テスト
    print("\n--- /lol-replays/v1/metadata/{id} ---")
    url = f'https://127.0.0.1:{rd.port}/lol-replays/v1/metadata/{numeric_id}'
    resp = requests.get(url, headers=headers, verify=False)
    print(f"Status: {resp.status_code}")
    print(f"Response: {resp.text[:500] if resp.text else 'Empty'}")
    
    # rofls一覧取得テスト
    print("\n--- /lol-replays/v1/rofls ---")
    url = f'https://127.0.0.1:{rd.port}/lol-replays/v1/rofls'
    resp = requests.get(url, headers=headers, verify=False)
    print(f"Status: {resp.status_code}")
    if resp.status_code == 200:
        rofls = resp.json()
        print(f"登録されているrofl数: {len(rofls)}")
        if rofls:
            print(f"最初の1件: {rofls[0] if isinstance(rofls, list) else list(rofls.keys())[0]}")

=== LCU API 接続情報 ===
Port: 57328
Token: 38FfaxIF9OorCQ42BQaZ...

=== 接続テスト ===
System builds: 200
  → LCU API接続OK

=== リプレイAPIテスト ===
REPLAY_DIR内のroflファイル数: 190

テスト対象: JP1-555620750.rofl
Game ID: JP1-555620750
Numeric ID: 555620750

--- /lol-replays/v1/metadata/{id} ---
Status: 200
Response: {"downloadProgress":100,"gameId":555620750,"state":"watch"}

--- /lol-replays/v1/rofls ---
Status: 404


In [3]:
# [3] 処理対象のroflファイル一覧を取得
# REPLAY_DIR内の全roflファイルを処理対象とします

import os

# REPLAY_DIRの存在確認
print(f"REPLAY_DIR: {REPLAY_DIR}")
print(f"  存在: {os.path.exists(REPLAY_DIR)}")

if not os.path.exists(REPLAY_DIR):
    raise FileNotFoundError(f"REPLAY_DIRが存在しません: {REPLAY_DIR}")

# REPLAY_DIRから全roflファイルを取得
rofl_files = [f for f in os.listdir(REPLAY_DIR) if f.endswith('.rofl')]
print(f"処理対象のroflファイル数: {len(rofl_files)}")

# 処理済みファイルをロード（再開機能）
def load_progress(log_path):
    """処理済みファイルの記録を読み込む"""
    if os.path.exists(log_path):
        with open(log_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    return {}

def save_progress(log_path, rofl_file, status):
    """処理結果を記録に追加"""
    progress = load_progress(log_path)
    progress[rofl_file] = {"status": status, "timestamp": str(os.popen('echo %date% %time%').read().strip())}
    with open(log_path, 'w', encoding='utf-8') as f:
        json.dump(progress, f, ensure_ascii=False, indent=2)

# 既存の進捗を確認
progress = load_progress(PROGRESS_LOG)
processed_files = [f for f, info in progress.items() if info.get("status") == "success"]
pending_files = [f for f in rofl_files if f not in processed_files]

print(f"\n=== 進捗状況 ===")
print(f"処理済み: {len(processed_files)}")
print(f"未処理: {len(pending_files)}")

REPLAY_DIR: C:\Users\lapis\Documents\League of Legends\Replays
  存在: True
処理対象のroflファイル数: 190

=== 進捗状況 ===
処理済み: 26
未処理: 164


In [4]:
# [4] 単体テスト - 1ファイルのみでリプレイ起動をテスト
# セル[6]の本番実行前にこのセルでテストしてください

import time

# rsのLCU認証情報を確認
print(f"=== ReplayScraper LCU設定 ===")
print(f"rs.lcu_port: {rs.lcu_port}")
print(f"rs.lcu_token: {rs.lcu_token[:15] if rs.lcu_token else 'None'}...")
print()

if not pending_files:
    print("未処理ファイルがありません。セル[3]を再実行してください。")
else:
    test_file = pending_files[0]
    game_id = test_file.replace('.rofl', '')
    
    print(f"=== 単体テスト（動的gameLength） ===")
    print(f"テスト対象: {test_file}")
    print(f"Game ID: {game_id}")
    print(f"キャプチャ: 1分～試合終了まで（自動検出）")
    print("3秒後に開始...")
    time.sleep(3)
    
    try:
        # end=None で試合終了まで自動キャプチャ
        rs.run_client_lcu(
            gameId=game_id,
            start=0,      # 1分から開始
            end=None,      # Noneで試合終了まで自動キャプチャ
            speed=8,
            paused=False,
            team="All",
            remove_fog_of_war=True
        )
        print(f"✓ テスト成功！")
    except Exception as e:
        print(f"✗ テスト失敗: {e}")

=== ReplayScraper LCU設定 ===
rs.lcu_port: 57328
rs.lcu_token: 38FfaxIF9OorCQ4...

=== 単体テスト（動的gameLength） ===
テスト対象: JP1-555620750.rofl
Game ID: JP1-555620750
キャプチャ: 1分～試合終了まで（自動検出）
3秒後に開始...
Replay launched: JP1-555620750
Waiting for replay client process...
Replay client process detected
Waiting for game to initialize...
Waiting for Replay API...
Replay API returned: 404
Replay API returned: 404
Replay API returned: 404
Replay API returned: 404
Replay API is ready
Game length detected: 1497.2s (25.0min)
Frame timestamps saved to: C:\dataset_20260105\JP1-555620750\frame_timestamps.csv
Captured 1536 frames for JP1-555620750
✓ テスト成功！


In [5]:
# [6] 自動バッチ処理 - 全roflファイルの再生＆キャプチャ
# 動的にgameLengthを取得し、試合終了まで自動キャプチャします
# エラー時はスキップして次へ進み、中断後も再開可能です

import time

# 処理設定
START_TIME = 0          # 1分からキャプチャ開始
REPLAY_SPEED = 8         # 再生速度

print(f"=== 自動バッチ処理開始（動的gameLength対応） ===")
print(f"処理対象: {len(pending_files)} ファイル")
print(f"キャプチャ設定: {START_TIME//60}分～試合終了（自動検出）, 速度{REPLAY_SPEED}x")
print(f"進捗ログ: {PROGRESS_LOG}")
print()

# 未処理ファイルを順次処理
for rofl_file in tqdm(pending_files, desc="Processing replays"):
    game_id = rofl_file.replace('.rofl', '')  
    
    try:
        print(f"\n[{rofl_file}] {START_TIME//60}分～試合終了をキャプチャ")
        
        # LCU API経由でリプレイ起動＆キャプチャ（end=Noneで自動検出）
        rs.run_client_lcu(
            gameId=game_id,
            start=START_TIME,
            end=None,  # Noneで試合終了まで自動キャプチャ
            speed=REPLAY_SPEED,
            paused=False,
            team="All",
            remove_fog_of_war=True
        )
        
        # 成功を記録
        save_progress(PROGRESS_LOG, rofl_file, "success")
        print(f"✓ {rofl_file} 完了")
        
    except Exception as e:
        # 失敗を記録してスキップ
        error_msg = str(e)
        save_progress(PROGRESS_LOG, rofl_file, f"failed: {error_msg}")
        print(f"✗ {rofl_file} 失敗: {error_msg}")
        
        # リプレイプロセスが残っている場合はクリーンアップ待ち
        time.sleep(5)
        continue

print(f"\n=== 処理完了 ===")
# 最終結果を表示
final_progress = load_progress(PROGRESS_LOG)
success_count = len([f for f, info in final_progress.items() if info.get("status") == "success"])
failed_count = len([f for f, info in final_progress.items() if "failed" in str(info.get("status", ""))])
print(f"成功: {success_count}, 失敗: {failed_count}")

=== 自動バッチ処理開始（動的gameLength対応） ===
処理対象: 164 ファイル
キャプチャ設定: 0分～試合終了（自動検出）, 速度8x
進捗ログ: C:\dataset_20260105\progress.json



Processing replays:   0%|          | 0/164 [00:00<?, ?it/s]


[JP1-555620750.rofl] 0分～試合終了をキャプチャ
Replay launched: JP1-555620750
Waiting for replay client process...
Replay client process detected
Waiting for game to initialize...
Waiting for Replay API...
Replay API returned: 404
Replay API returned: 404
Replay API is ready
Game length detected: 1497.2s (25.0min)
Frame timestamps saved to: C:\dataset_20260105\JP1-555620750\frame_timestamps.csv


Processing replays:   1%|          | 1/164 [03:27<9:22:46, 207.15s/it]

Captured 1585 frames for JP1-555620750
✓ JP1-555620750.rofl 完了

[JP1-555622520.rofl] 0分～試合終了をキャプチャ
Replay launched: JP1-555622520
Waiting for replay client process...
Replay client process detected
Waiting for game to initialize...
Waiting for Replay API...
Replay API returned: 404
Replay API returned: 404
Replay API returned: 404
Replay API is ready
Game length detected: 2101.4s (35.0min)
Frame timestamps saved to: C:\dataset_20260105\JP1-555622520\frame_timestamps.csv


...

Captured 1493 frames for JP1-556165177
✓ JP1-556165177.rofl 完了

[JP1-556174070.rofl] 0分～試合終了をキャプチャ
Replay launched: JP1-556174070
Waiting for replay client process...
Replay client process detected
Waiting for game to initialize...
Waiting for Replay API...
Replay API returned: 404
Replay API returned: 404
Replay API is ready
Game length detected: 1509.2s (25.2min)
Frame timestamps saved to: C:\dataset_20260105\JP1-556174070\frame_timestamps.csv


Processing replays:  60%|█████▉    | 98/164 [5:53:24<4:07:29, 225.00s/it]

Captured 1557 frames for JP1-556174070
✓ JP1-556174070.rofl 完了

[JP1-556183001.rofl] 0分～試合終了をキャプチャ
Replay launched: JP1-556183001
Waiting for replay client process...
Replay client process detected
Waiting for game to initialize...
Waiting for Replay API...
Replay API returned: 404
Replay API returned: 404
Replay API is ready
Game length detected: 1215.1s (20.3min)


Processing replays:  60%|█████▉    | 98/164 [5:54:22<3:58:39, 216.97s/it]


KeyboardInterrupt: 